In [1]:
from PIL import Image
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import PIL
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
import torch.autograd as autograd
import torchvision.utils
import torch.utils.data
import torchvision.models as models
import torchvision.transforms as transforms
import os
import sys
import numpy as np
import cv2
from scipy import misc
from matplotlib import pyplot as plt
# sys.path.append('yourdir/pretrained-models.pytorch') # if needed



In [2]:
batch_size = 1
num_epochs = 5
learning_rate = 0.01

In [3]:
class CDATA(torch.utils.data.Dataset): # Extend PyTorch's Dataset class
    def __init__(self, root_dir, train, transform=None):
        # root_dir  - the root directory of the dataset
        # train     - a boolean parameter representing whether to return the training set or the test set
        # transform - the transforms to be applied on the images before returning them
        #
        # In this function store the parameters in instance variables and make a mapping
        # from images to labels and keep it as an instance variable. Make sure to check which
        # dataset is required; train or test; and create the mapping accordingly.
        if train:
            cwd=root_dir+"/train"
        else :
            cwd=root_dir+"/test"
        sub_dir=os.listdir(cwd)
#         print(sub_dir)
#         input()
        self.imgAdd=[]
        self.custom_transform=transform
        self.imgAdd = [(cwd+"/"+image) for image in os.listdir(cwd)]
        self.LABEL=[]
#         print((self.imgAdd[0]))
        
# #             print(len(self.LABEL))
#         print(((self.imgAdd[0].split(".")[0]).split("/")[-1])[1:])
#         input()
    def __len__(self):
        # return the size of the dataset (total number of images) as an integer
        # this should be rather easy if you created a mapping in __init__
        return len(self.imgAdd)
    def __getitem__(self, idx):
        lbl=[]
        salImg = cv2.imread(self.imgAdd[idx],0)
        rawDir = "/home/vijayraj/Workspace/Acads/EE698K/Project/dataset/dataset1/rawImgs/"
        name_rawImg = rawDir+((self.imgAdd[idx].split(".")[0]).split("/")[-1])[1:]+".jpg" 
#         salImg = cv2.imread(salDir+file,0)
        origImg = cv2.imread(name_rawImg)
        imgS = cv2.resize(salImg,(150,150))
        imgO1 = cv2.resize(origImg,(150,150))
    
        W=150
        H=150
        numPatches = 5

        YcoorP,XcoorP = np.nonzero(imgS>255*0.9)
        YcoorP = YcoorP[YcoorP>21]
        XcoorP = XcoorP[XcoorP>21]
        YcoorP = YcoorP[YcoorP<H-21]
        XcoorP = XcoorP[XcoorP<W-21]
#         print(XcoorP.size)
#         input()
        XcoorP = XcoorP[np.random.randint(XcoorP.shape[0], size=numPatches)]
        YcoorP = YcoorP[np.random.randint(YcoorP.shape[0], size=numPatches)]

        YcoorN,XcoorN = np.nonzero(imgS<255*0.1)
        YcoorN = YcoorN[YcoorN>21]
        XcoorN = XcoorN[XcoorN>21]
        YcoorN = YcoorN[YcoorN<H-21]
        XcoorN = XcoorN[XcoorN<W-21]
        XcoorN = XcoorN[np.random.randint(XcoorN.shape[0], size=numPatches)]
        YcoorN = YcoorN[np.random.randint(YcoorN.shape[0], size=numPatches)]

        label =[]
        labelP =[]
        labelN =[]
        scale = [(150,150),(250,250),(400,400)]
        imgO2 = cv2.resize(origImg,scale[1])
        imgO3 = cv2.resize(origImg,scale[2])
        imgArray = [imgO1,imgO2,imgO3]
        for k in range(3):
#             print("k = ",k)
#             print("scale[k] = ",scale[k])
            ycoorP = (YcoorP*(scale[k][0]/150)).astype(np.int16)
            xcoorP = (XcoorP*(scale[k][0]/150)).astype(np.int16)
            ycoorN = (YcoorN*(scale[k][0]/150)).astype(np.int16)
            xcoorN = (XcoorN*(scale[k][0]/150)).astype(np.int16)
            if k==2:
                print(xcoorP.min(),xcoorP.max())
            stack = []
            imgO = imgArray[k]
            for i in range(5):
                
                patchP1 = imgO[ycoorP[i]-21:ycoorP[i]+21,xcoorP[i]-21:xcoorP[i]+21]
                cv2.imwrite("temPatchP.png",patchP1)
#                 if k==2:
#                     print("patchp1.shape = ",patchP1.shape)
#                     input()
                patchP1 = PIL.Image.open("temPatchP.png")
                patchP2 = PIL.ImageOps.mirror(patchP1)
                
                patchN1 = imgO[ycoorN[i]-21:ycoorN[i]+21,xcoorN[i]-21:xcoorN[i]+21]
                cv2.imwrite("temPatchN.png",patchN1)
                patchN1 = PIL.Image.open("temPatchN.png")
                patchN2 = PIL.ImageOps.mirror(patchN1)

                if self.custom_transform is not None:
                    patchP1 = self.custom_transform(patchP1).unsqueeze_(0)
                    patchP2 = self.custom_transform(patchP2).unsqueeze_(0)
                    patchN1 = self.custom_transform(patchN1).unsqueeze_(0)
                    patchN2 = self.custom_transform(patchN2).unsqueeze_(0)
#                 print("patchP1.shape = ",patchP1.size())
#                 input()
                if i==0:
                    stack = torch.cat((patchP1,patchP2,patchN1,patchN2),0)
                    
#                     label.append(0)
#                     print("stack.shape = ",stack.size())
#                     input()
                else :
                    stack = torch.cat((stack,patchP1,patchP2,patchN1,patchN2),0)
                if k==0:
                    label.append(1)
                    label.append(1)
                    label.append(0)
                    label.append(0)
#             print("stack.shape = ",stack.size())
#             print('lbl = ',label)
#             input()
            stack.unsqueeze_(0)
            if k==0:
                kstack = stack
            else:
                kstack = torch.cat((kstack,stack),0)

#         print("patch,shape = ",kstack.shape)
#         print("label len = ",len(label))
#         input()    

        return kstack,label
        

In [4]:
composed_transform = transforms.Compose([transforms.Scale((42,42)),transforms.ToTensor()])
train_dataset = CDATA(root_dir='/home/vijayraj/Workspace/Acads/EE698K/Project/dataset/dataset1/salImgs', train=True, transform=composed_transform) # Supply proper root_dir
test_dataset = CDATA(root_dir='/home/vijayraj/Workspace/Acads/EE698K/Project/dataset/dataset1/salImgs', train=False, transform=composed_transform) # Supply proper root_dir

# Let's check the size of the datasets, if implemented correctly they should be 16854 and 1870 respectively
print('Size of train dataset: %d' % len(train_dataset))
print('Size of test dataset: %d'% len(test_dataset))

# Create loaders for the dataset
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Let's look at one batch of train and test images
def imshow(img):
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    
train_dataiter = iter(train_loader)
train_images, train_labels = train_dataiter.next()
# print("Train images")
# imshow(torchvision.utils.make_grid(train_images))
# # input()
# test_dataiter = iter(test_loader)
# test_images, test_labels = test_dataiter.next()
# print("Test images")
# imshow(torchvision.utils.make_grid(test_images))

Size of train dataset: 100
Size of test dataset: 20
178 192


In [5]:
class MRCNN(nn.Module): # Extend PyTorch's Module class
    def __init__(self):
        super(MRCNN, self).__init__() # Must call super __init__()
        
        # Define the layers of the network here
        # There should be 17 total layers as evident from the diagram
        # The parameters and names for the layers are provided in the diagram
        # The variable names have to be the same as the ones in the diagram
        # Otherwise, the weights will not load
        self.conv1 = nn.Conv2d(3,96, kernel_size=7, stride=1,padding=0, bias=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2,padding=0)
        self.conv2 = nn.Conv2d(96,160, kernel_size=3, stride=1,padding=0, bias=True)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2,padding=0)
        self.conv3 = nn.Conv2d(160,288,kernel_size=3, stride=1, padding=0)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(2592, 512)
        self.fc2 = nn.Linear(2592, 512)
        self.fc3 = nn.Linear(2592, 512)
        self.fc4 = nn.Linear(1536,1)
        self.seq1 = nn.Sequential(self.conv1,self.maxpool1,
                                 self.conv2,self.maxpool2,
                                 self.conv3,self.maxpool3)
        self.seq2 = nn.Sequential(self.conv1,self.maxpool1,
                                 self.conv2,self.maxpool2,
                                 self.conv3,self.maxpool3)
        self.seq3 = nn.Sequential(self.conv1,self.maxpool1,
                                 self.conv2,self.maxpool2,
                                 self.conv3,self.maxpool3)
        
        
    def forward(self, x):
        # Here you have to define the forward pass
        # Make sure you take care of the skip connections
        x1 = x[:,0,:,:,:]
        x2 = x[:,1,:,:,:]
        x3 = x[:,2,:,:,:]
        out1 = self.seq1(x1)
        out2 = self.seq2(x2)
        out3 = self.seq3(x3)
        out1 = out1.view(-1, out1.numel())
        out2 = out2.view(-1, out2.numel())
        out3 = out3.view(-1, out3.numel())
#         print(out1.size())
#         out = tempout.view(-1, tempout.numel())
#         print(out2.size())
#         print(out3.size())
#         input()
        out1 = self.fc1(out1)
        out2 = self.fc2(out2)
        out3 = self.fc3(out3)
        out  = torch.cat((out1,out2,out3),1)
#         print("out.size = ",out.size())
        out = self.fc4(out)
        return out
    
    # conv2d(input, weight, bias, stride, padding, dilation, groups)
    # x = torch.cat(x_1, x_2, [dimension]) returns a Tensor x which is the concatenation of Tensors x_1 and x_2 along dimension dimension.

In [6]:
model_2 = MRCNN()
if torch.cuda.is_available():
    print("GPU accesible")
#     vgg16=vgg16.cuda(0)
    model_2=model_2.cuda(0)
# model_2 = nn.DataParallel(model_2, device_ids=[0, 1])

GPU accesible


In [7]:
criterion = nn.CrossEntropyLoss() # Define cross-entropy loss
# optimizer_vgg16 = optim.Adam(vgg16.parameters(), lr=learning_rate)# Use Adam optimizer, use learning_rate hyper parameter
optimizer_model_2 =optim.Adam(model_2.parameters(), lr=learning_rate) # Use Adam optimizer, use learning_rate hyper parameter

In [8]:


for epoch in range(num_epochs):
    running_loss = 0
    epoch_loss = 0
    count = 0
    print("epoch = ",epoch)
    for i, data in enumerate(train_loader, 0):
        # get the inputs
    #             print(i)
    ### assume that data is in the form of 10 rgb images i.e. 10*3*H*W 
        count+=1
        inputs, labels = data
        print("data.size = ",inputs.size())
        input()
#         labels=labels.long()
        # zero the parameter gradients
        for j in range(20):
#             ip1 = inputs[:,0,i,:,:,:]
#             ip2 = inputs[:,1,i,:,:,:]
#             ip3 = inputs[:,2,i,:,:,:]
            ip = inputs[:,:,i,:,:,:]
            
            ip = Variable(ip).cuda()
            label = labels[i].long()
            label = Variable(label).cuda() 
            
            optimizer_model_2.zero_grad()

#             ip1,ip2,ip3, label = Variable(ip1).cuda(),Variable(ip2).cuda(),Variable(ip3).cuda(), Variable(label).cuda()
#             output1 = model_2(ip1)
#                     = 
            output = model_2(ip)
#             print("output.size = ",output)
#             input()
#         # forward + backward + optimize
#             if img_indx ==0 :
#                 output = resnet18(img)
#             else :
#                 output = np.vstack((output,resnet18(img))
        
            loss = criterion(output, label)
#         print('loss = ',loss)
#         input()
            loss.backward()
            optimizer_model_2.step()

        # print statistics
            running_loss +=loss
            loss_numpy=loss.cpu().data.numpy()        
            epoch_loss +=loss_numpy
    #             if count%100==0:
    #                 print("count = ",count)
        if i % 500 == 499:    # print every 2000 mini-batches
            print('[%d, %5d]  '%
                  (epoch + 1, i + 1, ))
            print("running loss = ",running_loss/500)
            running_loss = 0.0
            count=0

    epoch_lossArray.append(epoch_loss)

epoch =  0
208 218
data.size =  torch.Size([1, 3, 20, 3, 42, 42])



RuntimeError: cuda runtime error (59) : device-side assert triggered at /opt/conda/conda-bld/pytorch_1503968623488/work/torch/lib/THC/generated/../generic/THCTensorMathPointwise.cu:313

In [ ]:
ip.size()

In [ ]:
H = 400
Ylist = np.arange(0,400-42)
Xlist = np.arange(0,400-42)
xcoor = np.random.randint(42,400-42,10)
ycoor = np.random.randint(42,400-42,10)

for i in range(10):
    Xlist[xcoor[i]-21:xcoor[i]+21] = 0
    Ylist[ycoor[i]-21:ycoor[i]+21] = 0
Xlist = np.unique(Xlist)
Ylist = np.unique(Ylist)
Xlist = Xlist[Xlist>=21]
# for i in range(10):
#     print(np.isin(np.arange(xcoor[i]-21,xcoor[i]+21),Xlist))
# plt.plot(Xlist)
# plt.show()
print(Xlist)


In [ ]:

salDir = "/home/vijayraj/Workspace/Acads/EE698K/Project/dataset/dataset1/salImgs/"
rawDir = "/home/vijayraj/Workspace/Acads/EE698K/Project/dataset/dataset1/rawImgs/"
imgfiles=[]
imgfiles2=[]
imgfiles += [each for each in os.listdir(salDir) if each.endswith('.jpg')]
imgfiles2 += [each for each in os.listdir("/home/vijayraj/Workspace/Acads/EE698K/Project/dataset/dataset1/rawImgs") if each.endswith('.jpg')]
print(imgfiles[0].split(".")[0][1:])

# input()
count = 0
numPatches=5
for file in imgfiles:
    salImg = cv2.imread(salDir+file,0)
    origImg = cv2.imread(rawDir+file.split(".")[0][1:]+".jpg")
    imgS = cv2.resize(salImg,(150,150))
    imgO1 = cv2.resize(origImg,(150,150))
#     cv2.imshow("orig",origImg)
#     for j in [(400,400),(250,250),(150,150)]:
#         print(j)
    W=150
        
#         print(imgS.shape)
    
    YcoorP,XcoorP = np.nonzero(imgS>255*0.9)
    print(XcoorP.size)
    input()
    YcoorP = YcoorP[YcoorP>21]
    XcoorP = XcoorP[XcoorP>21]
    print(XcoorP.size)
    YcoorP = YcoorP[YcoorP<H-21]
    XcoorP = XcoorP[XcoorP<W-21]
    print(XcoorP.size)
    input()
    XcoorP = XcoorP[np.random.randint(XcoorP.shape[0], size=numPatches)]
    YcoorP = YcoorP[np.random.randint(YcoorP.shape[0], size=numPatches)]
    
    YcoorN,XcoorN = np.nonzero(imgS<255*0.1)
    YcoorN = YcoorN[YcoorN>21]
    XcoorN = XcoorN[XcoorN>21]
    YcoorN = YcoorN[YcoorN<H-21]
    XcoorN = XcoorN[XcoorN<W-21]
    XcoorN = XcoorN[np.random.randint(XcoorN.shape[0], size=numPatches)]
    YcoorN = YcoorN[np.random.randint(YcoorN.shape[0], size=numPatches)]

    label =[]
    labelP =[]
    labelN =[]
    scale = [(150,150),(250,250),(400,400)]
    imgO2 = cv2.resize(origImg,scale[1])
    imgO3 = cv2.resize(origImg,scale[2])
    imgArray = [imgO1,imgO2,imgO3]
    for k in range(3):
        
        ycoorP = (YcoorP*scale[k][0]/150).astype(np.int8)
        xcoorP = (XcoorP*scale[k][0]/150).astype(np.int8)
        ycoorN = (YcoorN*scale[k][0]/150).astype(np.int8)
        xcoorN = (XcoorN*scale[k][0]/150).astype(np.int8)
        stack = []
        imgO = imgArray[k]
        for i in range(5):
            
            patchP1 = imgO[ycoorP[i]-21:ycoorP[i]+21,xcoorP[i]-21:xcoorP[i]+21]
            patchP2 = cv2.flip(patchP1,0)
            patchN1 = imgO[ycoorN[i]-21:ycoorN[i]+21,xcoorN[i]-21:xcoorN[i]+21]
            patchN2 = cv2.flip(patchN1,0)
            patchN1 = patchN1[np.newaxis,...]
            patchN2 = patchN2[np.newaxis,...]
            patchP1 = patchP1[np.newaxis,...]
            patchP2 = patchP2[np.newaxis,...]
            if i==0:
                stack = np.concatenate((patchP1,patchP2),axis = 0)
                stack = np.concatenate((stack,patchN1),axis = 0)
                stack = np.concatenate((stack,patchN2),axis = 0)
                label.append(1)
                label.append(1)
                label.append(0)
                label.append(0)
            else :
                stack = np.concatenate((stack,patchP1),axis = 0)
                stack = np.concatenate((stack,patchP2),axis = 0)
                stack = np.concatenate((stack,patchN1),axis = 0)
                stack = np.concatenate((stack,patchN2),axis = 0)
                label.append(1)
                label.append(1)
                label.append(0)
                label.append(0)
        stack = (np.concatenate((Pstack,Nstack),axis= 0))[np.newaxis,...]
        if k==0:
            kstack = stack
        else:
            kstack = np.concatenate((kstack,stack),axis = 0)
            
    print("patch,shape = ",kstack.shape)
    print("label len = ",len(label))
    input()
#             if count<=6480:
#                 saveDir = "/home/vijayraj/Workspace/Acads/EE698K/Project/dataset/train/"
#             else :
#                 saveDir = "/home/vijayraj/Workspace/Acads/EE698K/Project/dataset/test/"
                
#             cv2.imwrite(saveDir + "/1/"+"patchP_"+str(count)+".png",patchP1)
#             cv2.imwrite(saveDir + "/0/"+"patchN_"+str(count)+".png",patchN1)
#             cv2.imwrite(saveDir + "/1/"+"patchP_"+str(count+1)+".png",patchP2)
#             cv2.imwrite(saveDir + "/0/"+"patchN_"+str(count+1)+".png",patchN2)
#             count= count+2
    

In [ ]:
5112*3


In [ ]:
a = np.arange(25)
a= a[a>15]

In [ ]:
a